In [1]:
import pandas as pd
import numpy as np
import seaborn as sns;sns.set()
import matplotlib.pyplot as plt
import warnings
from all_funcs import util
from all_funcs.analysis import chi2_cal, ks_cal, chi2_count, ks_count,ks_not_pass,NIHSS_constraint_check
# pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
warnings.filterwarnings("ignore")

In [2]:
df=pd.read_csv("./dataset/df_noOutliner_ana.csv",index_col=0)
df, imp_mode, imp_mean=util.FeatureArrange(df)
## reduce redundant features which can be assembled
df=df.drop(['NIHTotal','THD_ID','cortical_CT', 'subcortical_CT',
              'circulation_CT', 'CT_find', 'watershed_CT', 'Hemorrhagic_infarct_CT',
              'CT_left', 'CT_right',],axis=1)

In [3]:
## ads-gan
df_ads_gan = pd.read_csv("./dataset/output_dataset/ADS-GAN_models.csv",index_col=0)
df_ads_gan_xtest = pd.read_csv("./dataset/output_dataset/ADS-GAN_XTEST.csv",index_col=0)
## + constraint
df_constraint = pd.read_csv("./dataset/output_dataset/ADS-GAN-constraint_models.csv",index_col=0)
## + constraint + condition
df_constraint_cond = pd.read_csv("./dataset/output_dataset/cADS-GAN_.csv",index_col=0)

In [4]:
#(16)
num = ['BT_NM', 'HR_NM', 'RR_NM', 'HB_NM', 'HCT_NM', 'PLATELET_NM', 'WBC_NM',
       'PTT1_NM', 'PTT2_NM', 'PTINR_NM', 'ER_NM', 'BUN_NM', 'CRE_NM', 'BMI',
       'age', 'PPD',] 
#(46)
cat = [ 
        'THDA_FL', 'THDH_FL', 'THDI_FL', 'THDAM_FL', 'THDV_FL',
        'THDE_FL', 'THDM_FL', 'THDR_FL', 'THDP_FL', 'THDOO_FL', 'Gender',
        'cortical_ACA_ctr', 'cortical_MCA_ctr', 'subcortical_ACA_ctr',
        'subcortical_MCA_ctr', 'PCA_cortex_ctr', 'thalamus_ctr',
        'brainstem_ctr', 'cerebellum_ctr', 'Watershed_ctr',
        'Hemorrhagic_infarct_ctr', 'cortical_ACA_ctl', 'cortical_MCA_ctl',
        'subcortical_ACA_ctl', 'subcortical_MCA_ctl', 'PCA_cortex_ctl',
        'thalamus_ctl', 'brainstem_ctl', 'cerebellum_ctl', 'Watershed_ctl',
        'Hemorrhagic_infarct_ctl', 'NIHS_1a_in', 'NIHS_1b_in', 'NIHS_1c_in',
        'NIHS_2_in', 'NIHS_3_in', 'NIHS_4_in', 'NIHS_5aL_in', 'NIHS_5bR_in',
        'NIHS_6aL_in', 'NIHS_6bR_in', 'NIHS_7_in', 'NIHS_8_in', 'NIHS_9_in',
        'NIHS_10_in', 'NIHS_11_in',
    ]
# (15)
NIHSS_col = [
        'NIHS_1a_in', 'NIHS_1b_in', 'NIHS_1c_in',
        'NIHS_2_in', 'NIHS_3_in', 'NIHS_4_in', 'NIHS_5aL_in', 'NIHS_5bR_in',
        'NIHS_6aL_in', 'NIHS_6bR_in', 'NIHS_7_in', 'NIHS_8_in', 'NIHS_9_in',
        'NIHS_10_in', 'NIHS_11_in'
]

# WGAN-GP in ADS-GAN

## ADS-GAN ks test

In [5]:
ks_rs=ks_cal(df_ads_gan,df,num)
# print(len(ks_rs))
pd.DataFrame(ks_rs).T.sort_values('pvalue')

,statistic,pvalue,tail_greater,tail_pvalue_great,tail_less,tail_pvalue_less
PTINR_NM,0.456677,4.20212e-137,True,1.47685e-137,True,4.77991e-123
CRE_NM,0.417053,2.05208e-114,True,1.50568e-63,True,7.43615e-115
BT_NM,0.290769,7.81293e-56,True,3.12107e-56,True,2.87924e-52
PTT1_NM,0.24711,1.9169e-40,True,7.92001e-41,True,1.29188e-13
HR_NM,0.182942,2.33171e-22,True,0.0001874,True,1.01231e-22
BMI,0.167778,7.12944e-19,True,8.92584e-05,True,3.13168e-19
WBC_NM,0.161465,1.64135e-17,True,0.0499425,True,7.24501e-18
RR_NM,0.15847,6.96752e-17,True,3.56787e-08,True,3.08263e-17
PTT2_NM,0.132005,7.60737e-12,True,3.43518e-12,True,2.835e-06
HB_NM,0.129493,2.04986e-11,True,9.23734e-10,True,9.27431e-12


## ADS-GAN chisquare test

In [6]:
chi2_rs=chi2_cal(df_ads_gan,df,df,cat)
pd.DataFrame(chi2_rs).T.sort_values('pvalue')[:10]

,statistic,pvalue
cerebellum_ctr,2.19,0.1389
NIHS_1c_in,3.74,0.1539
NIHS_7_in,3.44,0.1791
Hemorrhagic_infarct_ctr,1.40,0.2370
NIHS_11_in,2.80,0.2469
THDH_FL,1.26,0.2620
cerebellum_ctl,0.92,0.3363
THDE_FL,0.88,0.3482
THDR_FL,0.71,0.3994
Gender,0.65,0.4201


## ADS-GAN NIHSS constraint test

In [7]:
total_pass, res_detail = NIHSS_constraint_check(df_ads_gan,NIHSS_col)
print("total_pass: ",total_pass)
res_detail

total_pass:  True


,BT_NM,HR_NM,RR_NM,HB_NM,HCT_NM,PLATELET_NM,WBC_NM,PTT1_NM,PTT2_NM,PTINR_NM,ER_NM,BUN_NM,CRE_NM,BMI,age,PPD,THDA_FL,THDH_FL,THDI_FL,THDAM_FL,THDV_FL,THDE_FL,THDM_FL,THDR_FL,THDP_FL,THDOO_FL,Gender,cortical_ACA_ctr,cortical_MCA_ctr,subcortical_ACA_ctr,subcortical_MCA_ctr,PCA_cortex_ctr,thalamus_ctr,brainstem_ctr,cerebellum_ctr,Watershed_ctr,Hemorrhagic_infarct_ctr,cortical_ACA_ctl,cortical_MCA_ctl,subcortical_ACA_ctl,subcortical_MCA_ctl,PCA_cortex_ctl,thalamus_ctl,brainstem_ctl,cerebellum_ctl,Watershed_ctl,Hemorrhagic_infarct_ctl,NIHS_1a_in,NIHS_1b_in,NIHS_1c_in,NIHS_2_in,NIHS_3_in,NIHS_4_in,NIHS_5aL_in,NIHS_5bR_in,NIHS_6aL_in,NIHS_6bR_in,NIHS_7_in,NIHS_8_in,NIHS_9_in,NIHS_10_in,NIHS_11_in,elapsed_class
158,37.0,83.0,16.0,12.0,30.0,259.0,16.0,25.0,25.0,1.0,96.0,13.0,1.0,20.0,49.0,31.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,1.0,2.0,2.0,1.0,3.0,3.0,4.0,0.0,0.0,3.0,2.0,0.0,1.0
808,38.0,67.0,30.0,16.0,50.0,437.0,18.0,44.0,26.0,1.0,150.0,7.0,1.0,31.0,28.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,2.0,2.0,1.0,3.0,3.0,4.0,3.0,4.0,4.0,0.0,2.0,3.0,0.0,2.0,1.0
230,37.0,72.0,28.0,17.0,45.0,520.0,24.0,47.0,37.0,1.0,211.0,23.0,5.0,29.0,30.0,112.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,3.0,2.0,2.0,0.0,0.0,3.0,4.0,4.0,4.0,4.0,0.0,2.0,3.0,2.0,1.0,1.0
362,36.0,69.0,21.0,13.0,31.0,163.0,12.0,47.0,28.0,1.0,72.0,25.0,2.0,19.0,45.0,84.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,2.0,2.0,2.0,3.0,3.0,4.0,4.0,4.0,4.0,0.0,2.0,3.0,2.0,0.0,0.0


In [8]:
df_ads_gan.shape

(896, 63)

# ADS-GAN-constraint

In [9]:
## kstest
ks_rs=ks_cal(df_constraint,df,num)
pd.DataFrame(ks_rs).T.sort_values('pvalue')

,statistic,pvalue,tail_greater,tail_pvalue_great,tail_less,tail_pvalue_less
PTINR_NM,0.456677,4.20212e-137,True,1.47685e-137,True,4.77991e-123
CRE_NM,0.362366,1.76411e-86,True,1.16345e-55,True,6.66827e-87
BT_NM,0.288536,5.49861e-55,True,2.20035e-55,True,4.32051e-53
PTT1_NM,0.269432,5.32534e-48,True,2.16267e-48,True,1.69585e-09
ER_NM,0.194574,3.08802e-25,False,None,True,1.32868e-25
PTT2_NM,0.177686,4.07196e-21,True,0.00167639,True,1.77502e-21
RR_NM,0.154338,4.89973e-16,True,2.17471e-16,True,7.98781e-11
HB_NM,0.15287,9.67605e-16,True,4.2995e-16,True,1.5693e-09
WBC_NM,0.135795,1.64422e-12,True,0.00635451,True,7.40295e-13
BMI,0.124109,1.60877e-10,True,7.30895e-11,True,1.5911e-10


In [10]:
## chi2-test
chi2_rs=chi2_cal(df_constraint,df,df,cat)
pd.DataFrame(chi2_rs).T.sort_values('pvalue')[:10]

,statistic,pvalue
THDI_FL,1.87,0.1710
cerebellum_ctl,1.77,0.1837
THDAM_FL,1.40,0.2370
PCA_cortex_ctr,1.37,0.2421
NIHS_8_in,2.54,0.2804
THDE_FL,0.88,0.3482
NIHS_3_in,3.22,0.3589
NIHS_6aL_in,4.26,0.3724
THDR_FL,0.71,0.3994
THDOO_FL,0.51,0.4732


In [11]:
## NIHSS constraint
total_pass, res_detail = NIHSS_constraint_check(df_constraint, NIHSS_col)
print("total_pass: ",total_pass)
res_detail

total_pass:  True


,BT_NM,HR_NM,RR_NM,HB_NM,HCT_NM,PLATELET_NM,WBC_NM,PTT1_NM,PTT2_NM,PTINR_NM,ER_NM,BUN_NM,CRE_NM,BMI,age,PPD,THDA_FL,THDH_FL,THDI_FL,THDAM_FL,THDV_FL,THDE_FL,THDM_FL,THDR_FL,THDP_FL,THDOO_FL,Gender,cortical_ACA_ctr,cortical_MCA_ctr,subcortical_ACA_ctr,subcortical_MCA_ctr,PCA_cortex_ctr,thalamus_ctr,brainstem_ctr,cerebellum_ctr,Watershed_ctr,Hemorrhagic_infarct_ctr,cortical_ACA_ctl,cortical_MCA_ctl,subcortical_ACA_ctl,subcortical_MCA_ctl,PCA_cortex_ctl,thalamus_ctl,brainstem_ctl,cerebellum_ctl,Watershed_ctl,Hemorrhagic_infarct_ctl,NIHS_1a_in,NIHS_1b_in,NIHS_1c_in,NIHS_2_in,NIHS_3_in,NIHS_4_in,NIHS_5aL_in,NIHS_5bR_in,NIHS_6aL_in,NIHS_6bR_in,NIHS_7_in,NIHS_8_in,NIHS_9_in,NIHS_10_in,NIHS_11_in,elapsed_class
773,37.0,123.0,21.0,14.0,43.0,200.0,10.0,31.0,26.0,1.0,297.0,20.0,1.0,23.0,63.0,40.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,2.0,2.0,2.0,4.0,0.0,4.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0
271,36.0,58.0,20.0,14.0,41.0,320.0,15.0,38.0,26.0,1.0,178.0,19.0,1.0,19.0,80.0,116.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,3.0,2.0,2.0,2.0,2.0,3.0,4.0,4.0,2.0,3.0,0.0,2.0,3.0,2.0,2.0,0.0
62,36.0,60.0,21.0,8.0,25.0,402.0,16.0,35.0,41.0,2.0,174.0,56.0,3.0,23.0,81.0,97.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,3.0,2.0,2.0,2.0,3.0,3.0,4.0,4.0,4.0,4.0,0.0,0.0,3.0,2.0,2.0,1.0
140,37.0,63.0,14.0,6.0,15.0,640.0,22.0,19.0,26.0,1.0,408.0,68.0,4.0,28.0,81.0,99.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,3.0,2.0,2.0,0.0,0.0,3.0,4.0,4.0,4.0,4.0,0.0,0.0,3.0,2.0,2.0,1.0


# ADS-GAN-constraint-condition

In [12]:
## kstest
ks_rs=ks_cal(df_constraint_cond,df,num)
pd.DataFrame(ks_rs).T.sort_values('pvalue')

,statistic,pvalue,tail_greater,tail_pvalue_great,tail_less,tail_pvalue_less
PTINR_NM,0.454445,9.04289e-136,True,3.18364e-136,True,4.77991e-123
BT_NM,0.368894,1.31272e-89,True,4.93707e-90,True,1.49975e-27
CRE_NM,0.360134,2.01112e-85,True,1.79821e-41,True,7.61506e-86
PTT1_NM,0.185782,4.80192e-23,True,2.08018e-23,True,1.03573e-05
BMI,0.163172,7.1141e-18,True,3.13608e-18,True,2.89838e-05
RR_NM,0.159919,3.4745e-17,True,1.5355e-17,True,1.3336e-08
WBC_NM,0.150304,3.13015e-15,True,0.000108123,True,1.39363e-15
HB_NM,0.150249,3.2094e-15,True,3.46138e-11,True,1.42897e-15
PTT2_NM,0.138624,5.09705e-13,True,0.000245165,True,2.28989e-13
HCT_NM,0.108746,3.55575e-08,False,None,True,1.63472e-08


In [13]:
## chi2-test
chi2_rs=chi2_cal(df_constraint_cond,df,df,cat)
pd.DataFrame(chi2_rs).T.sort_values('pvalue')[:10]

,statistic,pvalue
THDAM_FL,2.48,0.1152
NIHS_2_in,4.28,0.1177
THDM_FL,1.42,0.2336
cortical_MCA_ctl,1.26,0.2608
PCA_cortex_ctl,1.16,0.2816
THDE_FL,0.88,0.3482
brainstem_ctl,0.57,0.4515
THDA_FL,0.50,0.4811
THDI_FL,0.49,0.4819
cerebellum_ctl,0.48,0.4881


In [14]:
## NIHSS constraint
total_pass, res_detail = NIHSS_constraint_check(df_constraint_cond, NIHSS_col)
print("total_pass: ",total_pass)
res_detail

total_pass:  True


,BT_NM,HR_NM,RR_NM,HB_NM,HCT_NM,PLATELET_NM,WBC_NM,PTT1_NM,PTT2_NM,PTINR_NM,ER_NM,BUN_NM,CRE_NM,BMI,age,PPD,THDA_FL,THDH_FL,THDI_FL,THDAM_FL,THDV_FL,THDE_FL,THDM_FL,THDR_FL,THDP_FL,THDOO_FL,Gender,cortical_ACA_ctr,cortical_MCA_ctr,subcortical_ACA_ctr,subcortical_MCA_ctr,PCA_cortex_ctr,thalamus_ctr,brainstem_ctr,cerebellum_ctr,Watershed_ctr,Hemorrhagic_infarct_ctr,cortical_ACA_ctl,cortical_MCA_ctl,subcortical_ACA_ctl,subcortical_MCA_ctl,PCA_cortex_ctl,thalamus_ctl,brainstem_ctl,cerebellum_ctl,Watershed_ctl,Hemorrhagic_infarct_ctl,NIHS_1a_in,NIHS_1b_in,NIHS_1c_in,NIHS_2_in,NIHS_3_in,NIHS_4_in,NIHS_5aL_in,NIHS_5bR_in,NIHS_6aL_in,NIHS_6bR_in,NIHS_7_in,NIHS_8_in,NIHS_9_in,NIHS_10_in,NIHS_11_in,elapsed_class
